In [18]:
import pandas as pd
import datetime as dt
import pandasql as ps
import os
import zipfile as zp
from datetime import datetime
import matplotlib.pyplot as plt
import plotly.express as px

#from kaggle.api.kaggle_api_extended import KaggleApi
#api = KaggleApi()
#api.authenticate()

#dataset_name=api.dataset_list_files('prakrutchauhan/indian-candidates-for-general-election-2019').files

DIR=r'/home/anuj/Downloads/'
ZIp='archive(2).zip'

os.chdir(DIR)
zf = zp.ZipFile(f'{ZIp}')

dfs={}
for name in zf.namelist():
    df_name= name.replace('.csv','')
    dfs[df_name]=pd.read_csv(zf.open(name,mode='r'),encoding='latin')

df=dfs['GOI']
df.columns=df.columns.str.replace(' ','_')
df.columns=df.columns.str.replace('\n','_')
df.columns=df.columns.str.replace('(','')
df.columns=df.columns.str.replace(')','')
df.columns=df.columns.str.replace('-','')
df.columns=df.columns.str.replace('/','')



#os.chdir(r'C:\Users\anuj\Documents\My Tableau Repository\Datasources')
#df.to_csv('indian_elections.csv',index=False)

/tmp/ipykernel_23027/2190296164.py:30: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.

/tmp/ipykernel_23027/2190296164.py:31: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.



In [19]:
df2=df.melt(id_vars=['Category','Country_States_Union_Territories_Name'],value_name='Literacy_Rate',var_name='Type')
df2=df2.rename(columns={"Country_States_Union_Territories_Name":"STATE_UT"})
df2['Year']=df2['Type'].apply(lambda x: x.split('__')[2])
df2['Typ']=df2['Type'].apply(lambda x: x.split('__')[1])
del df2['Type']


In [3]:
df2.isnull().isnull().sum()
df2[df2.duplicated()]

,Category,STATE_UT,Literacy_Rate,Year,Typ


In [5]:
# NUMBER OF UNIQUE VALUES

for col in df2.columns:
    print(col,df2[col].nunique())

Category 2
STATE_UT 35
Literacy_Rate 163
Year 2
Typ 3


### AVG LITERACY RATE BY STATE

In [34]:
q1=df2[(df2['Typ']!='Total')&(df2['STATE_UT']!='INDIA')].groupby(['STATE_UT','Year','Typ']).agg({"Literacy_Rate":"mean"}).reset_index().sort_values(by=['Year','Literacy_Rate'],ascending=[False,True])
q1['Literacy_Rate_lab']=q1['Literacy_Rate'].apply(lambda x: "{:.1%}".format(x/100))
fig=px.bar(q1,x='Literacy_Rate',y='STATE_UT',orientation='h',facet_col='Year',barmode='group',color='Typ',hover_data=q1.columns,text='Literacy_Rate_lab',height=2000)
fig.update_layout({"showlegend":True,"title":"Literacy Rate by State and Year","title_font_size":20,"font_family":"Courier New","title_font_color":"red","legend_title_font_color":"green"})
fig.write_image(format='png',file='Literacy Rate by State.png',validate=True)
fig.show()

In [31]:
q2=df2[(df2['Typ']!='Total')&(df2['STATE_UT']!='INDIA')].groupby(['STATE_UT','Year','Typ']).agg({"Literacy_Rate":"mean"}).sort_values(by='Literacy_Rate',ascending=True).reset_index()
q2=q2.set_index(['STATE_UT','Typ','Year'])['Literacy_Rate'].unstack().reset_index().rename(columns={"2001":"last_census","2011":"latest_census"})
q2['chng']=q2['latest_census']-q2['last_census']
q2=q2.sort_values(by='chng',ascending=True)
q2['chng_lab']=q2['chng'].apply(lambda x: "{:.1%}".format(x/100))


fig=px.bar(q2,x='chng',y='STATE_UT',orientation='h',color='Typ',facet_col='Typ',hover_data=q2.columns,text='chng_lab',height=2000)
fig.update_layout({"title":"States with change in Literacy Rate from Census is 2001","title_font_size":20,"font_family":"Courier New","title_font_color":"red","legend_title_font_color":"green"})
fig.write_image(format='png',file='States with change in Literacy Rate.png',validate=True)
fig.show()



### Difference b/w Literacy Rates Max and Minimum

In [36]:
q3=df2[(df2['Typ']!='Total')&(df2['STATE_UT']!='INDIA')].groupby(['STATE_UT','Year']).agg({"Literacy_Rate":"mean"}).sort_values(by='Literacy_Rate',ascending=True).reset_index()
q3=q3.set_index(['STATE_UT','Year'])['Literacy_Rate'].unstack().reset_index().rename(columns={"2001":"last_census","2011":"latest_census"})
q3.groupby('STATE_UT').agg({"latest_census":"mean"}).rank(ascending=False)
q3=pd.concat([q3,q3.groupby('STATE_UT').agg({"latest_census":"mean"}).rank(ascending=False).reset_index().rename(columns={"latest_census":"LR_Rank","STATE_UT":"STATE"})],axis=1)[['STATE_UT','latest_census','LR_Rank']]

ans=q3.sort_values(by='LR_Rank',ascending=True).head(1).append(q3.sort_values(by='LR_Rank',ascending=True).tail(1))
ans[['latest_census']].reset_index(drop=True).diff()

Year,latest_census
0,NaN
1,-25.7


In [61]:
q4=df2.groupby(['Category','Typ','Year']).agg({"Literacy_Rate":"mean"}).reset_index()
#q4=q4.set_index(['Category','Typ','Year'])['Literacy_Rate'].unstack().reset_index().rename(columns={"2001":"last_census","2011":"latest_census"})

q4['Literacy_Rate_lab']= q4['Literacy_Rate'].apply(lambda x: "{:.1%}".format(x/100))
fig =px.bar(q4,x='Literacy_Rate',y='Year',facet_col='Category',facet_row='Typ',color='Typ',text='Literacy_Rate_lab')
fig.update_layout({"title":"Summary of Literacy Rate","title_font_size":20,"font_family":"Courier New","title_font_color":"blue","legend_title_font_color":"green"})
fig.write_image(format='png',file='Summary of Literacy Rate.png',validate=True)
fig.show()

In [60]:
df2.head()

,Category,STATE_UT,Literacy_Rate,Year,Typ
0,Country,INDIA,64.8,2001,Total
1,State,Andhra Pradesh,60.5,2001,Total
2,State,Arunachal Pradesh,54.3,2001,Total
3,State,Assam,63.3,2001,Total
4,State,Bihar,47.0,2001,Total
